# Stable Chrome + Chromedriver Setup for Colab

In [3]:
%%bash
# Remove any old Chrome/Chromedriver
rm -rf /usr/bin/chromedriver /usr/local/bin/chromedriver

# Install Google Chrome
wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
apt-get -y install ./google-chrome-stable_current_amd64.deb

# Get Chrome version
CHROME_VERSION=$(google-chrome --version | cut -d " " -f3 | cut -d "." -f1)
echo "Installed Chrome version: $CHROME_VERSION"

# Download matching ChromeDriver
DRIVER_VERSION=$(wget -qO- "https://googlechromelabs.github.io/chrome-for-testing/LATEST_RELEASE_${CHROME_VERSION}")
echo "Matching Driver version: $DRIVER_VERSION"

wget -q "https://storage.googleapis.com/chrome-for-testing-public/${DRIVER_VERSION}/linux64/chromedriver-linux64.zip"
unzip -q chromedriver-linux64.zip
mv chromedriver-linux64/chromedriver /usr/bin/chromedriver
chmod +x /usr/bin/chromedriver

# Verify installation
google-chrome --version
chromedriver --version



Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.9 MB/130 MB of archives.
After this operation, 438 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan1 amd64 1.3.204.1-2 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 mesa-vulkan-drivers amd64 23.2.1-1ubuntu3.1~22.04.3 [10.7 MB]
Get:3 /content/google-chrome-stable_current_amd64.deb google-chrome-stable amd64 139.0.7258.127-1 [119 MB]
Fetched 10.9 MB in 1s (7,769 kB/s)
Selecting previously unselected package libvulkan1:amd64.
(Reading database ... 126380 files and directories currently installed.)
Preparing to unpack .../libvulkan1_1.3.204.1-2_amd64.deb ...
Unpack

In [4]:
# checking
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")   # Run headless (no GUI)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Point to chromedriver
service = Service("/usr/bin/chromedriver")

# Start browser
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get("https://www.google.com")

print("✅ Page title:", driver.title)

driver.quit()


✅ Page title: Google


# code

In [1]:
# ================================================
# 📌 LinkedIn Job Scraper
# ================================================
#
# -----------------------------------------------
# Dependencies: selenium>=4.18, webdriver-manager, undetected-chromedriver,
# pandas, beautifulsoup4, lxml, tenacity, python-dateutil
# -----------------------------------------------
!pip install -q selenium==4.18.1 webdriver-manager undetected-chromedriver pandas beautifulsoup4 lxml tenacity python-dateutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 61.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 25.5 MB/s eta 0:00:00


In [2]:
import os
import time
import random
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [5]:
# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India","Bengaluru, Karnataka, India"],
    "result_limit": 30,          # Max jobs per keyword-location combo
    "fetch_descriptions": False,  # Set True for full job descriptions
    "dedupe_on": ["Link"]
}

In [6]:
# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

In [7]:
# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    if li_at:
        driver.get("https://www.linkedin.com")
        driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
        driver.refresh()
        print("[INFO] Logged in with li_at cookie")
    elif username and password:
        driver.get("https://www.linkedin.com/login")
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        print("[INFO] Logged in with username/password")
    else:
        print("[WARN] No login provided → scraping public jobs only")

In [8]:
# ==================== SCRAPER ====================
def build_search_url(keyword: str, location: str, start: int = 0) -> str:
    return (
        f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"
    )

def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool) -> List[Dict]:
    jobs = []
    start = 0
    while len(jobs) < limit:
        url = build_search_url(keyword, location, start)
        driver.get(url)
        time.sleep(random.uniform(2, 4))

        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            try:
                title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
                company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
                loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
                link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
                date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

                job = {
                    "Title": title,
                    "Company": company,
                    "Location": loc,
                    "Date Posted": date_posted,
                    "Link": link,
                    "Job Type": None,
                    "Seniority": None,
                    "Applicants": None,
                    "Job Function": None,
                    "Job Description": None,
                    "Scraped At": datetime.utcnow().isoformat()
                }

                if fetch_desc:
                    try:
                        card.click()
                        WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-description"))
                        )
                        soup = BeautifulSoup(driver.page_source, "lxml")
                        desc = soup.select_one("div.jobs-description__content")
                        job["Job Description"] = desc.get_text(" ", strip=True) if desc else None
                    except Exception:
                        pass

                jobs.append(job)
                if len(jobs) >= limit:
                    break
            except Exception:
                continue

        start += len(cards)
        if len(cards) == 0:
            break

    return jobs

In [9]:
# ==================== MAIN ====================
def main(CONFIG):
    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"])
            all_jobs.extend(jobs)
            print(f"[INFO] Fetched {len(jobs)} jobs for {kw} in {loc}")

    driver.quit()

    df = pd.DataFrame(all_jobs)
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])

    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = f"linkedin_jobs_{ts}.csv"
    json_path = f"linkedin_jobs_{ts}.json"
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    print(f"[SUMMARY] Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    print(f"Saved → {csv_path}, {json_path}")
    display(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)


[WARN] No login provided → scraping public jobs only
[INFO] Fetched 30 jobs for Data Analyst in Pune, Maharashtra, India
[INFO] Fetched 30 jobs for Data Analyst in Bengaluru, Karnataka, India
[INFO] Fetched 30 jobs for Software Engineer in Pune, Maharashtra, India
[INFO] Fetched 30 jobs for Software Engineer in Bengaluru, Karnataka, India
[SUMMARY] Total scraped: 120, Unique after dedupe: 120
Saved → linkedin_jobs_20250816_095000.csv, linkedin_jobs_20250816_095000.json


,Title,Company,Location,Date Posted,Link,Job Type,Seniority,Applicants,Job Function,Job Description,Scraped At
0,Data Analyst,bp,"Pune, Maharashtra, India",2025-07-04,https://in.linkedin.com/jobs/view/data-analyst...,None,None,None,None,None,2025-08-16T09:49:40.864192
1,Data Analyst,Wipro,"Pune, Maharashtra, India",2025-07-18,https://in.linkedin.com/jobs/view/data-analyst...,None,None,None,None,None,2025-08-16T09:49:40.914435
2,Data Analyst,Michelin,"Pune, Maharashtra, India",2025-08-07,https://in.linkedin.com/jobs/view/data-analyst...,None,None,None,None,None,2025-08-16T09:49:40.963192
3,Healthcare Data Analyst,Persistent Systems,"Pune, Maharashtra, India",2025-08-12,https://in.linkedin.com/jobs/view/healthcare-d...,None,None,None,None,None,2025-08-16T09:49:41.017789
4,Data Analyst,The Lubrizol Corporation,"Pune, Maharashtra, India",2025-08-02,https://in.linkedin.com/jobs/view/data-analyst...,None,None,None,None,None,2025-08-16T09:49:41.084224


(120, 11)


In [10]:
import os
import time
import random
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India","Bengaluru, Karnataka, India"],
    "result_limit": 30,          # Max jobs per keyword-location combo
    "fetch_descriptions": False,  # Set True for full job descriptions
    "dedupe_on": ["Link"]
}

# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    if li_at:
        driver.get("https://www.linkedin.com")
        driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
        driver.refresh()
        print("[INFO] Logged in with li_at cookie")
    elif username and password:
        driver.get("https://www.linkedin.com/login")
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        print("[INFO] Logged in with username/password")
    else:
        print("[WARN] No login provided → scraping public jobs only")

# ==================== SCRAPER ====================
def build_search_url(keyword: str, location: str, start: int = 0) -> str:
    return (
        f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"
    )

def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool) -> List[Dict]:
    jobs = []
    start = 0
    while len(jobs) < limit:
        url = build_search_url(keyword, location, start)
        driver.get(url)
        time.sleep(random.uniform(2, 4))

        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            try:
                title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
                company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
                loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
                link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
                date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

                job = {
                    "Title": title,
                    "Company": company,
                    "Location": loc,
                    "Date Posted": date_posted,
                    "Link": link,
                    "Job Type": None,
                    "Seniority": None,
                    "Applicants": None,
                    "Job Function": None,
                    "Job Description": None,
                    "Scraped At": datetime.utcnow().isoformat()
                }

                if fetch_desc:
                    try:
                        card.click()
                        WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-description"))
                        )
                        soup = BeautifulSoup(driver.page_source, "lxml")
                        desc = soup.select_one("div.jobs-description__content")
                        job["Job Description"] = desc.get_text(" ", strip=True) if desc else None
                    except Exception:
                        pass

                jobs.append(job)
                if len(jobs) >= limit:
                    break
            except Exception:
                continue

        start += len(cards)
        if len(cards) == 0:
            break

    return jobs

# ==================== MAIN ====================
def main(CONFIG):
    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"])
            all_jobs.extend(jobs)
            print(f"[INFO] Fetched {len(jobs)} jobs for {kw} in {loc}")

    driver.quit()

    df = pd.DataFrame(all_jobs)
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])

    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = f"linkedin_jobs_{ts}.csv"
    json_path = f"linkedin_jobs_{ts}.json"
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    print(f"[SUMMARY] Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    print(f"Saved → {csv_path}, {json_path}")
    display(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)


[WARN] No login provided → scraping public jobs only
[INFO] Fetched 30 jobs for Data Analyst in Pune, Maharashtra, India
[INFO] Fetched 30 jobs for Data Analyst in Bengaluru, Karnataka, India
[INFO] Fetched 30 jobs for Software Engineer in Pune, Maharashtra, India
[INFO] Fetched 30 jobs for Software Engineer in Bengaluru, Karnataka, India
[SUMMARY] Total scraped: 120, Unique after dedupe: 120
Saved → linkedin_jobs_20250816_095104.csv, linkedin_jobs_20250816_095104.json


,Title,Company,Location,Date Posted,Link,Job Type,Seniority,Applicants,Job Function,Job Description,Scraped At
0,Data Analyst,bp,"Pune, Maharashtra, India",2025-07-04,https://in.linkedin.com/jobs/view/data-analyst...,None,None,None,None,None,2025-08-16T09:50:43.287567
1,Data Analyst,Wipro,"Pune, Maharashtra, India",2025-07-18,https://in.linkedin.com/jobs/view/data-analyst...,None,None,None,None,None,2025-08-16T09:50:43.337203
2,Data Analyst,Michelin,"Pune, Maharashtra, India",2025-08-07,https://in.linkedin.com/jobs/view/data-analyst...,None,None,None,None,None,2025-08-16T09:50:43.389981
3,Healthcare Data Analyst,Persistent Systems,"Pune, Maharashtra, India",2025-08-12,https://in.linkedin.com/jobs/view/healthcare-d...,None,None,None,None,None,2025-08-16T09:50:43.439184
4,Data Analyst,The Lubrizol Corporation,"Pune, Maharashtra, India",2025-08-02,https://in.linkedin.com/jobs/view/data-analyst...,None,None,None,None,None,2025-08-16T09:50:43.501025


(120, 11)


# try 1

In [11]:
import os
import time
import json
import logging
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India", "Bengaluru, Karnataka, India"],
    "result_limit": 30,          # Max jobs per keyword-location combo
    "fetch_descriptions": True,  # Fetch full job descriptions
    "dedupe_on": ["Link"],
    "output_dir": "outputs"
}

# ==================== LOGGER ====================
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO,
    datefmt="%H:%M:%S"
)

# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    try:
        if li_at:
            driver.get("https://www.linkedin.com")
            driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
            driver.refresh()
            logging.info("Logged in with li_at cookie ✅")
        elif username and password:
            driver.get("https://www.linkedin.com/login")
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
            driver.find_element(By.ID, "password").send_keys(password)
            driver.find_element(By.XPATH, "//button[@type='submit']").click()
            logging.info("Logged in with username/password ✅")
        else:
            logging.warning("No login provided → scraping public jobs only ⚠️")
    except Exception as e:
        logging.error(f"Login failed: {e}")

# ==================== SCRAPER ====================
def build_search_url(keyword: str, location: str, start: int = 0) -> str:
    return f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"

def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool) -> List[Dict]:
    jobs = []
    start = 0
    while len(jobs) < limit:
        url = build_search_url(keyword, location, start)
        driver.get(url)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.jobs-search__results-list li"))
            )
        except:
            logging.warning("No job cards found, stopping...")
            break

        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            try:
                title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
                company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
                loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
                link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
                date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

                job = {
                    "Title": title,
                    "Company": company,
                    "Location": loc,
                    "Date Posted": date_posted,
                    "Link": link,
                    "Job Type": None,
                    "Seniority": None,
                    "Applicants": None,
                    "Job Function": None,
                    "Job Description": None,
                    "Scraped At": datetime.utcnow().isoformat()
                }

                if fetch_desc:
                    try:
                        card.click()
                        WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-description"))
                        )
                        soup = BeautifulSoup(driver.page_source, "lxml")
                        desc = soup.select_one("div.jobs-description__content")
                        job["Job Description"] = desc.get_text(" ", strip=True) if desc else None

                        details = soup.select("ul.description__job-criteria-list li")
                        for d in details:
                            label = d.select_one("h3").text.strip()
                            value = d.select_one("span").text.strip()
                            if "Seniority" in label: job["Seniority"] = value
                            if "Employment" in label: job["Job Type"] = value
                            if "Job function" in label: job["Job Function"] = value
                            if "Applicants" in label: job["Applicants"] = value
                    except Exception:
                        pass

                jobs.append(job)
                if len(jobs) >= limit:
                    break
            except Exception:
                continue

        start += len(cards)
        if len(cards) == 0:
            break

    return jobs

# ==================== MAIN ====================
def main(CONFIG):
    os.makedirs(CONFIG["output_dir"], exist_ok=True)

    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"])
            all_jobs.extend(jobs)
            logging.info(f"Fetched {len(jobs)} jobs for {kw} in {loc}")

    driver.quit()

    df = pd.DataFrame(all_jobs)
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])

    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = os.path.join(CONFIG["output_dir"], f"linkedin_jobs_{ts}.csv")
    json_path = os.path.join(CONFIG["output_dir"], f"linkedin_jobs_{ts}.json")
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    logging.info(f"SUMMARY → Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    logging.info(f"Saved CSV → {csv_path}")
    logging.info(f"Saved JSON → {json_path}")

    print(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)


                     Title             Company                  Location  \
0             Data Analyst                  bp  Pune, Maharashtra, India   
1             Data Analyst               Wipro  Pune, Maharashtra, India   
2             Data Analyst            Michelin  Pune, Maharashtra, India   
3  Healthcare Data Analyst  Persistent Systems  Pune, Maharashtra, India   
4                                                                          

  Date Posted                                               Link Job Type  \
0  2025-07-04  https://in.linkedin.com/jobs/view/data-analyst...     None   
1  2025-07-18  https://in.linkedin.com/jobs/view/data-analyst...     None   
2  2025-08-07  https://in.linkedin.com/jobs/view/data-analyst...     None   
3  2025-08-12  https://in.linkedin.com/jobs/view/healthcare-d...     None   
4  2025-08-02  https://in.linkedin.com/jobs/view/data-analyst...     None   

  Seniority Applicants Job Function Job Description  \
0      None       None   

# try 2

In [12]:
import os
import time
import random
import json
import logging
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India", "Bengaluru, Karnataka, India"],
    "result_limit": 30,           # Max jobs per keyword-location combo
    "fetch_descriptions": True,   # Always fetch details
    "dedupe_on": ["Link"]
}

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    if li_at:
        driver.get("https://www.linkedin.com")
        driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
        driver.refresh()
        logging.info("Logged in with li_at cookie")
    elif username and password:
        driver.get("https://www.linkedin.com/login")
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        logging.info("Logged in with username/password")
    else:
        logging.warning("No login provided → scraping public jobs only")

# ==================== SCRAPER ====================
def build_search_url(keyword: str, location: str, start: int = 0) -> str:
    return f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"

def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool) -> List[Dict]:
    jobs = []
    start = 0
    while len(jobs) < limit:
        url = build_search_url(keyword, location, start)
        driver.get(url)
        time.sleep(random.uniform(1.5, 2.5))

        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            try:
                title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
                company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
                loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
                link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
                date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

                job = {
                    "Title": title,
                    "Company": company,
                    "Location": loc,
                    "Date Posted": date_posted,
                    "Link": link,
                    "Job Type": "",
                    "Seniority": "",
                    "Applicants": "",
                    "Job Function": "",
                    "Job Description": "",
                    "Scraped At": datetime.utcnow().isoformat()
                }

                if fetch_desc:
                    try:
                        driver.execute_script("arguments[0].click();", card)
                        WebDriverWait(driver, 6).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-details"))
                        )
                        soup = BeautifulSoup(driver.page_source, "lxml")

                        # Description
                        desc = soup.select_one("div.jobs-description__content")
                        job["Job Description"] = desc.get_text(" ", strip=True) if desc else ""

                        # Job criteria (Seniority, Employment, Function, Applicants)
                        for li in soup.select("ul.description__job-criteria-list li"):
                            label = li.select_one("h3")
                            value = li.select_one("span")
                            if not label or not value:
                                continue
                            label = label.get_text(strip=True)
                            value = value.get_text(strip=True)
                            if "Seniority" in label:
                                job["Seniority"] = value
                            elif "Employment" in label:
                                job["Job Type"] = value
                            elif "Job function" in label:
                                job["Job Function"] = value
                            elif "Applicants" in label:
                                job["Applicants"] = value
                    except Exception as e:
                        logging.debug(f"Detail extraction failed: {e}")

                jobs.append(job)
                logging.info(f"Scraped: {title} | {company} | {loc} | {date_posted}")
                if len(jobs) >= limit:
                    break
            except Exception as e:
                logging.debug(f"Card parse failed: {e}")
                continue

        start += len(cards)
        if len(cards) == 0:
            break

    return jobs

# ==================== MAIN ====================
def main(CONFIG):
    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"])
            all_jobs.extend(jobs)
            logging.info(f"Fetched {len(jobs)} jobs for {kw} in {loc}")

    driver.quit()

    df = pd.DataFrame(all_jobs).fillna("")
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])

    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = f"linkedin_jobs_{ts}.csv"
    json_path = f"linkedin_jobs_{ts}.json"
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    logging.info(f"SUMMARY → Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    logging.info(f"Saved → {csv_path}, {json_path}")
    print(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)


                     Title             Company                  Location  \
0             Data Analyst                  bp  Pune, Maharashtra, India   
1             Data Analyst               Wipro  Pune, Maharashtra, India   
2             Data Analyst            Michelin  Pune, Maharashtra, India   
3  Healthcare Data Analyst  Persistent Systems  Pune, Maharashtra, India   
4                                                                          

  Date Posted                                               Link Job Type  \
0  2025-07-04  https://in.linkedin.com/jobs/view/data-analyst...            
1  2025-07-18  https://in.linkedin.com/jobs/view/data-analyst...            
2  2025-08-07  https://in.linkedin.com/jobs/view/data-analyst...            
3  2025-08-12  https://in.linkedin.com/jobs/view/healthcare-d...            
4  2025-08-02  https://in.linkedin.com/jobs/view/data-analyst...            

  Seniority Applicants Job Function Job Description  \
0                        

# try 3

In [14]:
import os
import time
import random
import json
import logging
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India", "Bengaluru, Karnataka, India"],
    "result_limit": 30,           # Max jobs per keyword-location combo
    "fetch_descriptions": False,   # Toggle: fetch job descriptions (slower but richer data)
    "dedupe_on": ["Link"],

    #Filters
    "filters": {
        "remote": '2',             # None, "1" (remote), "2" (on-site), "3" (hybrid)
        "date_posted": 'r86400',        # None, "r86400" (past 24h), "r604800" (past week), "r2592000" (past month)
        "experience": None,         # 1=Internship, 2=Entry level, 3=Associate, 4=Mid-Senior, 5=Director, 6=Executive
        "employment_type": None     # 1=Full-time, 2=Part-time, 3=Contract, 4=Temporary, 5=Volunteer, 6=Internship, 7=Other
    }
}

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    if li_at:
        driver.get("https://www.linkedin.com")
        driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
        driver.refresh()
        logging.info("Logged in with li_at cookie")
    elif username and password:
        driver.get("https://www.linkedin.com/login")
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        logging.info("Logged in with username/password")
    else:
        logging.warning("No login provided → scraping public jobs only")

# ==================== SCRAPER ====================
def build_search_url(keyword: str, location: str, start: int = 0, filters: dict = None) -> str:
    base = f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"
    if filters:
        if filters.get("remote"):
            base += f"&f_WT={filters['remote']}"
        if filters.get("date_posted"):
            base += f"&f_TPR={filters['date_posted']}"
        if filters.get("experience"):
            base += f"&f_E={filters['experience']}"
        if filters.get("employment_type"):
            base += f"&f_JT={filters['employment_type']}"
    return base

def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool, filters: dict) -> List[Dict]:
    jobs = []
    start = 0
    while len(jobs) < limit:
        url = build_search_url(keyword, location, start, filters)
        driver.get(url)
        time.sleep(random.uniform(1.5, 2.5))

        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            try:
                title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
                company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
                loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
                link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
                date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

                job = {
                    "Title": title,
                    "Company": company,
                    "Location": loc,
                    "Date Posted": date_posted,
                    "Link": link,
                    "Job Type": "",
                    "Seniority": "",
                    "Applicants": "",
                    "Job Function": "",
                    "Job Description": "",
                    "Scraped At": datetime.utcnow().isoformat()
                }

                if fetch_desc:
                    try:
                        driver.execute_script("arguments[0].click();", card)
                        WebDriverWait(driver, 6).until(
                            EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-details"))
                        )
                        soup = BeautifulSoup(driver.page_source, "lxml")

                        # Description
                        desc = soup.select_one("div.jobs-description__content")
                        job["Job Description"] = desc.get_text(" ", strip=True) if desc else ""

                        # Job criteria (Seniority, Employment, Function, Applicants)
                        for li in soup.select("ul.description__job-criteria-list li"):
                            label = li.select_one("h3")
                            value = li.select_one("span")
                            if not label or not value:
                                continue
                            label = label.get_text(strip=True)
                            value = value.get_text(strip=True)
                            if "Seniority" in label:
                                job["Seniority"] = value
                            elif "Employment" in label:
                                job["Job Type"] = value
                            elif "Job function" in label:
                                job["Job Function"] = value
                            elif "Applicants" in label:
                                job["Applicants"] = value
                    except Exception as e:
                        logging.debug(f"Detail extraction failed: {e}")

                jobs.append(job)
                logging.info(f"Scraped: {title} | {company} | {loc} | {date_posted}")
                if len(jobs) >= limit:
                    break
            except Exception as e:
                logging.debug(f"Card parse failed: {e}")
                continue

        start += len(cards)
        if len(cards) == 0:
            break

    return jobs

# ==================== MAIN ====================
def main(CONFIG):
    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"], CONFIG["filters"])
            all_jobs.extend(jobs)
            logging.info(f"Fetched {len(jobs)} jobs for {kw} in {loc}")

    driver.quit()

    df = pd.DataFrame(all_jobs).fillna("")
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])

    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = f"linkedin_jobs_{ts}.csv"
    json_path = f"linkedin_jobs_{ts}.json"
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    logging.info(f"SUMMARY → Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    logging.info(f"Saved → {csv_path}, {json_path}")
    print(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)


                                               Title       Company  \
0                                     Data Scientist  Fusemachines   
1                                Azure Data Engineer    Bridgenext   
2       Director - Data Governance and AI Operations     SailPoint   
3  Freelance Software Developer (Python) - Qualit...      Mindrift   
4  Senior Software Engineer (Python/Go - Backend)...                 

                   Location Date Posted  \
0  Pune, Maharashtra, India  2025-08-15   
1  Pune, Maharashtra, India  2025-08-15   
2  Pune, Maharashtra, India  2025-08-16   
3  Pune, Maharashtra, India  2025-08-16   
4                            2025-08-15   

                                                Link Job Type Seniority  \
0  https://in.linkedin.com/jobs/view/data-scienti...                      
1  https://in.linkedin.com/jobs/view/azure-data-e...                      
2  https://in.linkedin.com/jobs/view/director-dat...                      
3  https://in.linked

# try 4

In [15]:
import os
import json
import time
import random
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India", "Bengaluru, Karnataka, India"],
    "result_limit": 30,          # Max jobs per keyword-location combo
    "fetch_descriptions": False, # Toggle to scrape full job descriptions

    # --- NEW FILTERS ---
    "filters": {
        "remote": None,          # options: None, "1" (remote), "2" (on-site), "3" (hybrid)
        "date_posted": None,     # options: None, "r86400" (past 24h), "r604800" (past week), etc.
        "experience": None,      # options: 1=Internship, 2=Entry, 3=Associate, etc.
        "employment_type": None, # options: 1=Full-time, 2=Part-time, etc.
    },

    "dedupe_on": ["Link"],       # Remove duplicates based on job link
    "save_chunk_size": 50        # Save after every N jobs
}

# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    driver.get("https://www.linkedin.com/login")

    if li_at:
        driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
        driver.refresh()
        print("[INFO] Logged in with li_at cookie")
    elif username and password:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        print("[INFO] Logged in with username/password")
    else:
        print("[WARN] No login → scraping public jobs only")

# ==================== URL BUILDER ====================
def build_search_url(keyword: str, location: str, start: int = 0, filters: dict = None) -> str:
    url = f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"
    if filters:
        if filters.get("remote"): url += f"&f_WT={filters['remote']}"
        if filters.get("date_posted"): url += f"&f_TPR={filters['date_posted']}"
        if filters.get("experience"): url += f"&f_E={filters['experience']}"
        if filters.get("employment_type"): url += f"&f_JT={filters['employment_type']}"
    return url

# ==================== JOB EXTRACTION ====================
def extract_job_card(card, fetch_desc: bool, driver) -> Dict:
    """Extracts job info from a single LinkedIn job card"""
    try:
        title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
        company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
        loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
        link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
        date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

        job = {
            "Title": title,
            "Company": company,
            "Location": loc,
            "Date Posted": date_posted,
            "Link": link,
            "Scraped At": datetime.utcnow().isoformat()
        }

        # Fetch job description if enabled
        if fetch_desc:
            try:
                card.click()
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-description"))
                )
                soup = BeautifulSoup(driver.page_source, "lxml")
                desc = soup.select_one("div.jobs-description__content")
                if desc:
                    job["Job Description"] = desc.get_text(" ", strip=True)
            except Exception:
                job["Job Description"] = None

        return job
    except Exception:
        return None

# ==================== SCRAPER ====================
def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool, filters: dict) -> List[Dict]:
    jobs = []
    seen_links = set()
    start = 0

    while len(jobs) < limit:
        url = build_search_url(keyword, location, start, filters)
        driver.get(url)

        # Smart wait instead of fixed sleep
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.jobs-search__results-list li"))
        )
        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            job = extract_job_card(card, fetch_desc, driver)
            if job and job["Link"] not in seen_links:
                jobs.append(job)
                seen_links.add(job["Link"])

            if len(jobs) >= limit:
                break

        start += len(cards)

    return jobs

# ==================== MAIN ====================
def main(CONFIG):
    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = f"linkedin_jobs_{ts}.csv"
    json_path = f"linkedin_jobs_{ts}.json"

    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"], CONFIG["filters"])
            all_jobs.extend(jobs)
            print(f"[INFO] Fetched {len(jobs)} jobs for {kw} in {loc}")

            # Save incrementally
            if len(all_jobs) >= CONFIG["save_chunk_size"]:
                df = pd.DataFrame(all_jobs)
                if CONFIG["dedupe_on"]:
                    df = df.drop_duplicates(subset=CONFIG["dedupe_on"])
                df.to_csv(csv_path, index=False)
                df.to_json(json_path, orient="records", indent=2)
                print(f"[AUTO-SAVE] Saved {len(df)} jobs → {csv_path}, {json_path}")

    driver.quit()

    # Final save
    df = pd.DataFrame(all_jobs)
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    print(f"[SUMMARY] Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    print(f"Saved → {csv_path}, {json_path}")
    print(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)


[WARN] No login → scraping public jobs only
[INFO] Fetched 30 jobs for Data Analyst in Pune, Maharashtra, India
[INFO] Fetched 30 jobs for Data Analyst in Bengaluru, Karnataka, India
[AUTO-SAVE] Saved 60 jobs → linkedin_jobs_20250816_102959.csv, linkedin_jobs_20250816_102959.json
[INFO] Fetched 30 jobs for Software Engineer in Pune, Maharashtra, India
[AUTO-SAVE] Saved 90 jobs → linkedin_jobs_20250816_102959.csv, linkedin_jobs_20250816_102959.json
[INFO] Fetched 30 jobs for Software Engineer in Bengaluru, Karnataka, India
[AUTO-SAVE] Saved 120 jobs → linkedin_jobs_20250816_102959.csv, linkedin_jobs_20250816_102959.json
[SUMMARY] Total scraped: 120, Unique after dedupe: 120
Saved → linkedin_jobs_20250816_102959.csv, linkedin_jobs_20250816_102959.json
                     Title                   Company  \
0             Data Analyst                        bp   
1             Data Analyst                     Wipro   
2             Data Analyst                  Michelin   
3  Healthcare Da

# try 5

In [17]:
import os
import json
import time
import random
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India", "Bengaluru, Karnataka, India"],
    "result_limit": 30,           # Max jobs per keyword-location combo
    "fetch_descriptions": False,  # Toggle to scrape full job descriptions

    # --- FILTERS ---
    "filters": {
        "remote": None,           # options: None, "1" (remote), "2" (on-site), "3" (hybrid)
        "date_posted": "r86400", # past week (None, r86400=24h, r604800=week, r2592000=month)
        "experience": None,       # options: 1=Internship, 2=Entry, 3=Associate, etc.
        "employment_type": None,  # options: 1=Full-time, 2=Part-time, etc.
    },

    "dedupe_on": ["Link"],        # Remove duplicates based on job link
    "save_chunk_size": 50,        # Save after every N jobs
    "max_retries": 3,             # Retry failed pages/cards N times
    "retry_delay": 3              # Seconds between retries
}

# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    driver.get("https://www.linkedin.com/login")

    if li_at:
        driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
        driver.refresh()
        print("[INFO] Logged in with li_at cookie")
    elif username and password:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        print("[INFO] Logged in with username/password")
    else:
        print("[WARN] No login → scraping public jobs only")

# ==================== URL BUILDER ====================
def build_search_url(keyword: str, location: str, start: int = 0, filters: dict = None) -> str:
    url = f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"
    if filters:
        if filters.get("remote"): url += f"&f_WT={filters['remote']}"
        if filters.get("date_posted"): url += f"&f_TPR={filters['date_posted']}"
        if filters.get("experience"): url += f"&f_E={filters['experience']}"
        if filters.get("employment_type"): url += f"&f_JT={filters['employment_type']}"
    return url

# ==================== JOB EXTRACTION ====================
def extract_job_card(card, fetch_desc: bool, driver, retries=CONFIG["max_retries"]) -> Dict:
    """Extracts job info from a single LinkedIn job card with retries"""
    for attempt in range(retries):
        try:
            title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
            company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
            loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
            link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
            date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

            job = {
                "Title": title,
                "Company": company,
                "Location": loc,
                "Date Posted": date_posted,
                "Link": link,
                "Scraped At": datetime.utcnow().isoformat()
            }

            # Fetch job description if enabled
            if fetch_desc:
                try:
                    card.click()
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-description"))
                    )
                    soup = BeautifulSoup(driver.page_source, "lxml")
                    desc = soup.select_one("div.jobs-description__content")
                    if desc:
                        job["Job Description"] = desc.get_text(" ", strip=True)
                except Exception:
                    job["Job Description"] = None

            return job
        except Exception:
            time.sleep(CONFIG["retry_delay"])
            print(f"[WARN] Job extraction retry {attempt+1}/{retries}")
    return None

# ==================== SCRAPER ====================
def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool, filters: dict) -> List[Dict]:
    jobs = []
    seen_links = set()
    start = 0

    while len(jobs) < limit:
        url = build_search_url(keyword, location, start, filters)

        # Retry page load
        for attempt in range(CONFIG["max_retries"]):
            try:
                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.jobs-search__results-list li"))
                )
                break
            except Exception:
                print(f"[WARN] Page load failed → retry {attempt+1}/{CONFIG['max_retries']}")
                time.sleep(CONFIG["retry_delay"])
        else:
            print("[ERROR] Skipping page due to repeated failures")
            break

        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            job = extract_job_card(card, fetch_desc, driver)
            if job and job["Link"] not in seen_links:
                jobs.append(job)
                seen_links.add(job["Link"])

            if len(jobs) >= limit:
                break

        start += len(cards)

    return jobs

# ==================== MAIN ====================
def main(CONFIG):
    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = f"linkedin_jobs_{ts}.csv"
    json_path = f"linkedin_jobs_{ts}.json"

    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"], CONFIG["filters"])
            all_jobs.extend(jobs)
            print(f"[INFO] Fetched {len(jobs)} jobs for {kw} in {loc}")

            # Save incrementally
            if len(all_jobs) >= CONFIG["save_chunk_size"]:
                df = pd.DataFrame(all_jobs)
                if CONFIG["dedupe_on"]:
                    df = df.drop_duplicates(subset=CONFIG["dedupe_on"])
                df.to_csv(csv_path, index=False)
                df.to_json(json_path, orient="records", indent=2)
                print(f"[AUTO-SAVE] Saved {len(df)} jobs → {csv_path}, {json_path}")

    driver.quit()

    # Final save
    df = pd.DataFrame(all_jobs)
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    print(f"[SUMMARY] Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    print(f"Saved → {csv_path}, {json_path}")
    print(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)


[WARN] No login → scraping public jobs only
[INFO] Fetched 30 jobs for Data Analyst in Pune, Maharashtra, India
[INFO] Fetched 30 jobs for Data Analyst in Bengaluru, Karnataka, India
[AUTO-SAVE] Saved 60 jobs → linkedin_jobs_20250816_103524.csv, linkedin_jobs_20250816_103524.json
[INFO] Fetched 30 jobs for Software Engineer in Pune, Maharashtra, India
[AUTO-SAVE] Saved 88 jobs → linkedin_jobs_20250816_103524.csv, linkedin_jobs_20250816_103524.json
[INFO] Fetched 30 jobs for Software Engineer in Bengaluru, Karnataka, India
[AUTO-SAVE] Saved 118 jobs → linkedin_jobs_20250816_103524.csv, linkedin_jobs_20250816_103524.json
[SUMMARY] Total scraped: 120, Unique after dedupe: 118
Saved → linkedin_jobs_20250816_103524.csv, linkedin_jobs_20250816_103524.json
                         Title                           Company  \
0                      Analyst                      IntegriChain   
1                    Associate                         PwC India   
2                 Data Analyst  GK H

# try 6

In [19]:
import os
import json
import time
import random
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India", "Bengaluru, Karnataka, India"],
    "result_limit": 30,           # Max jobs per keyword-location combo
    "fetch_descriptions": False,  # Toggle to scrape full job descriptions

    # --- FILTERS ---
    "filters": {
        "remote": None,           # options: None, "1" (remote), "2" (on-site), "3" (hybrid)
        "date_posted": "r86400", # past week (None, r86400=24h, r604800=week, r2592000=month)
        "experience": None,       # options: 1=Internship, 2=Entry, 3=Associate, etc.
        "employment_type": None,  # options: 1=Full-time, 2=Part-time, etc.
    },

    "dedupe_on": ["Link"],        # Remove duplicates based on job link
    "save_chunk_size": 50,        # Save after every N jobs
    "max_retries": 3,             # Retry failed pages/cards N times
    "retry_delay": 3,             # Seconds between retries

    # --- HUMAN-LIKE DELAY ---
    "random_delay": {
        "enabled": False,          # True = wait random seconds, False = skip waiting
        "min_sec": 2,             # Minimum seconds to wait
        "max_sec": 6              # Maximum seconds to wait
    }
}

# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    driver.get("https://www.linkedin.com/login")

    if li_at:
        driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
        driver.refresh()
        print("[INFO] Logged in with li_at cookie")
    elif username and password:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        print("[INFO] Logged in with username/password")
    else:
        print("[WARN] No login → scraping public jobs only")

# ==================== URL BUILDER ====================
def build_search_url(keyword: str, location: str, start: int = 0, filters: dict = None) -> str:
    url = f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"
    if filters:
        if filters.get("remote"): url += f"&f_WT={filters['remote']}"
        if filters.get("date_posted"): url += f"&f_TPR={filters['date_posted']}"
        if filters.get("experience"): url += f"&f_E={filters['experience']}"
        if filters.get("employment_type"): url += f"&f_JT={filters['employment_type']}"
    return url

# ==================== RANDOM DELAY ====================
def apply_random_delay():
    if CONFIG["random_delay"]["enabled"]:
        delay = random.uniform(CONFIG["random_delay"]["min_sec"], CONFIG["random_delay"]["max_sec"])
        print(f"[WAIT] Sleeping {delay:.2f} seconds (human-like delay)")
        time.sleep(delay)

# ==================== JOB EXTRACTION ====================
def extract_job_card(card, fetch_desc: bool, driver, retries=CONFIG["max_retries"]) -> Dict:
    """Extracts job info from a single LinkedIn job card with retries"""
    for attempt in range(retries):
        try:
            title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
            company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
            loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
            link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
            date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

            job = {
                "Title": title,
                "Company": company,
                "Location": loc,
                "Date Posted": date_posted,
                "Link": link,
                "Scraped At": datetime.utcnow().isoformat()
            }

            # Fetch job description if enabled
            if fetch_desc:
                try:
                    card.click()
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-description"))
                    )
                    soup = BeautifulSoup(driver.page_source, "lxml")
                    desc = soup.select_one("div.jobs-description__content")
                    if desc:
                        job["Job Description"] = desc.get_text(" ", strip=True)
                except Exception:
                    job["Job Description"] = None

            return job
        except Exception:
            time.sleep(CONFIG["retry_delay"])
            print(f"[WARN] Job extraction retry {attempt+1}/{retries}")
    return None

# ==================== SCRAPER ====================
def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool, filters: dict) -> List[Dict]:
    jobs = []
    seen_links = set()
    start = 0

    while len(jobs) < limit:
        url = build_search_url(keyword, location, start, filters)

        # Retry page load
        for attempt in range(CONFIG["max_retries"]):
            try:
                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.jobs-search__results-list li"))
                )
                break
            except Exception:
                print(f"[WARN] Page load failed → retry {attempt+1}/{CONFIG['max_retries']}")
                time.sleep(CONFIG["retry_delay"])
        else:
            print("[ERROR] Skipping page due to repeated failures")
            break

        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            job = extract_job_card(card, fetch_desc, driver)
            if job and job["Link"] not in seen_links:
                jobs.append(job)
                seen_links.add(job["Link"])

                # Apply random human-like delay
                apply_random_delay()

            if len(jobs) >= limit:
                break

        start += len(cards)

    return jobs

# ==================== MAIN ====================
def main(CONFIG):
    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = f"linkedin_jobs_{ts}.csv"
    json_path = f"linkedin_jobs_{ts}.json"

    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"], CONFIG["filters"])
            all_jobs.extend(jobs)
            print(f"[INFO] Fetched {len(jobs)} jobs for {kw} in {loc}")

            # Save incrementally
            if len(all_jobs) >= CONFIG["save_chunk_size"]:
                df = pd.DataFrame(all_jobs)
                if CONFIG["dedupe_on"]:
                    df = df.drop_duplicates(subset=CONFIG["dedupe_on"])
                df.to_csv(csv_path, index=False)
                df.to_json(json_path, orient="records", indent=2)
                print(f"[AUTO-SAVE] Saved {len(df)} jobs → {csv_path}, {json_path}")

    driver.quit()

    # Final save
    df = pd.DataFrame(all_jobs)
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    print(f"[SUMMARY] Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    print(f"Saved → {csv_path}, {json_path}")
    print(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)


[WARN] No login → scraping public jobs only
[INFO] Fetched 30 jobs for Data Analyst in Pune, Maharashtra, India
[INFO] Fetched 30 jobs for Data Analyst in Bengaluru, Karnataka, India
[AUTO-SAVE] Saved 60 jobs → linkedin_jobs_20250816_104542.csv, linkedin_jobs_20250816_104542.json
[INFO] Fetched 30 jobs for Software Engineer in Pune, Maharashtra, India
[AUTO-SAVE] Saved 88 jobs → linkedin_jobs_20250816_104542.csv, linkedin_jobs_20250816_104542.json
[INFO] Fetched 30 jobs for Software Engineer in Bengaluru, Karnataka, India
[AUTO-SAVE] Saved 118 jobs → linkedin_jobs_20250816_104542.csv, linkedin_jobs_20250816_104542.json
[SUMMARY] Total scraped: 120, Unique after dedupe: 118
Saved → linkedin_jobs_20250816_104542.csv, linkedin_jobs_20250816_104542.json
                                               Title  \
0                                            Analyst   
1                                          Associate   
2                                       Data Analyst   
3  Senior Analys

# FINALIZED

In [ ]:
import os
import json
import time
import random
import pandas as pd
from datetime import datetime
from typing import List, Dict

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [ ]:
# ==================== CONFIG ====================
CONFIG = {
    "keywords": ["Data Analyst", "Software Engineer"],
    "locations": ["Pune, Maharashtra, India", "Bengaluru, Karnataka, India"],
    "result_limit": 30,           # Max jobs per keyword-location combo
    "fetch_descriptions": False,  # Toggle to scrape full job descriptions

    # --- FILTERS ---
    "filters": {
        "remote": None,           # options: None, "1" (remote), "2" (on-site), "3" (hybrid)
        "date_posted": "r86400", # past week (None, r86400=24h, r604800=week, r2592000=month)
        "experience": None,       # options: 1=Internship, 2=Entry, 3=Associate, etc.
        "employment_type": None,  # options: 1=Full-time, 2=Part-time, etc.
    },

    "dedupe_on": ["Link"],        # Remove duplicates based on job link
    "save_chunk_size": 50,        # Save after every N jobs
    "max_retries": 3,             # Retry failed pages/cards N times
    "retry_delay": 3,             # Seconds between retries

    # --- HUMAN-LIKE DELAY ---
    "random_delay": {
        "enabled": False,          # True = wait random seconds, False = skip waiting
        "min_sec": 2,             # Minimum seconds to wait
        "max_sec": 6              # Maximum seconds to wait
    }
}

In [ ]:
# ==================== DRIVER SETUP ====================
def setup_driver() -> uc.Chrome:
    options = uc.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

In [ ]:
# ==================== AUTH ====================
def login_linkedin(driver):
    li_at = os.getenv("LI_AT")
    username = os.getenv("LINKEDIN_USER")
    password = os.getenv("LINKEDIN_PASS")

    driver.get("https://www.linkedin.com/login")

    if li_at:
        driver.add_cookie({"name": "li_at", "value": li_at, "domain": ".linkedin.com"})
        driver.refresh()
        print("[INFO] Logged in with li_at cookie")
    elif username and password:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys(username)
        driver.find_element(By.ID, "password").send_keys(password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        print("[INFO] Logged in with username/password")
    else:
        print("[WARN] No login → scraping public jobs only")

In [ ]:
# ==================== URL BUILDER ====================
def build_search_url(keyword: str, location: str, start: int = 0, filters: dict = None) -> str:
    url = f"https://www.linkedin.com/jobs/search/?keywords={keyword}&location={location}&start={start}"
    if filters:
        if filters.get("remote"): url += f"&f_WT={filters['remote']}"
        if filters.get("date_posted"): url += f"&f_TPR={filters['date_posted']}"
        if filters.get("experience"): url += f"&f_E={filters['experience']}"
        if filters.get("employment_type"): url += f"&f_JT={filters['employment_type']}"
    return url

In [ ]:
# ==================== RANDOM DELAY ====================
def apply_random_delay():
    if CONFIG["random_delay"]["enabled"]:
        delay = random.uniform(CONFIG["random_delay"]["min_sec"], CONFIG["random_delay"]["max_sec"])
        print(f"[WAIT] Sleeping {delay:.2f} seconds (human-like delay)")
        time.sleep(delay)

In [ ]:
# ==================== JOB EXTRACTION ====================
def extract_job_card(card, fetch_desc: bool, driver, retries=CONFIG["max_retries"]) -> Dict:
    """Extracts job info from a single LinkedIn job card with retries"""
    for attempt in range(retries):
        try:
            title = card.find_element(By.CSS_SELECTOR, "h3").text.strip()
            company = card.find_element(By.CSS_SELECTOR, "h4").text.strip()
            loc = card.find_element(By.CSS_SELECTOR, "span.job-search-card__location").text.strip()
            link = card.find_element(By.TAG_NAME, "a").get_attribute("href").split("?")[0]
            date_posted = card.find_element(By.CSS_SELECTOR, "time").get_attribute("datetime")

            job = {
                "Title": title,
                "Company": company,
                "Location": loc,
                "Date Posted": date_posted,
                "Link": link,
                "Scraped At": datetime.utcnow().isoformat()
            }

            # Fetch job description if enabled
            if fetch_desc:
                try:
                    card.click()
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "div.jobs-description"))
                    )
                    soup = BeautifulSoup(driver.page_source, "lxml")
                    desc = soup.select_one("div.jobs-description__content")
                    if desc:
                        job["Job Description"] = desc.get_text(" ", strip=True)
                except Exception:
                    job["Job Description"] = None

            return job
        except Exception:
            time.sleep(CONFIG["retry_delay"])
            print(f"[WARN] Job extraction retry {attempt+1}/{retries}")
    return None

In [ ]:
# ==================== SCRAPER ====================
def scrape_jobs(driver, keyword: str, location: str, limit: int, fetch_desc: bool, filters: dict) -> List[Dict]:
    jobs = []
    seen_links = set()
    start = 0

    while len(jobs) < limit:
        url = build_search_url(keyword, location, start, filters)

        # Retry page load
        for attempt in range(CONFIG["max_retries"]):
            try:
                driver.get(url)
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, "ul.jobs-search__results-list li"))
                )
                break
            except Exception:
                print(f"[WARN] Page load failed → retry {attempt+1}/{CONFIG['max_retries']}")
                time.sleep(CONFIG["retry_delay"])
        else:
            print("[ERROR] Skipping page due to repeated failures")
            break

        cards = driver.find_elements(By.CSS_SELECTOR, "ul.jobs-search__results-list li")
        if not cards:
            break

        for card in cards:
            job = extract_job_card(card, fetch_desc, driver)
            if job and job["Link"] not in seen_links:
                jobs.append(job)
                seen_links.add(job["Link"])

                # Apply random human-like delay
                apply_random_delay()

            if len(jobs) >= limit:
                break

        start += len(cards)

    return jobs

In [20]:
# ==================== MAIN ====================
def main(CONFIG):
    driver = setup_driver()
    login_linkedin(driver)

    all_jobs = []
    ts = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
    csv_path = f"linkedin_jobs_{ts}.csv"
    json_path = f"linkedin_jobs_{ts}.json"

    for kw in CONFIG["keywords"]:
        for loc in CONFIG["locations"]:
            jobs = scrape_jobs(driver, kw, loc, CONFIG["result_limit"], CONFIG["fetch_descriptions"], CONFIG["filters"])
            all_jobs.extend(jobs)
            print(f"[INFO] Fetched {len(jobs)} jobs for {kw} in {loc}")

            # Save incrementally
            if len(all_jobs) >= CONFIG["save_chunk_size"]:
                df = pd.DataFrame(all_jobs)
                if CONFIG["dedupe_on"]:
                    df = df.drop_duplicates(subset=CONFIG["dedupe_on"])
                df.to_csv(csv_path, index=False)
                df.to_json(json_path, orient="records", indent=2)
                print(f"[AUTO-SAVE] Saved {len(df)} jobs → {csv_path}, {json_path}")

    driver.quit()

    # Final save
    df = pd.DataFrame(all_jobs)
    if CONFIG["dedupe_on"]:
        df = df.drop_duplicates(subset=CONFIG["dedupe_on"])
    df.to_csv(csv_path, index=False)
    df.to_json(json_path, orient="records", indent=2)

    print(f"[SUMMARY] Total scraped: {len(all_jobs)}, Unique after dedupe: {len(df)}")
    print(f"Saved → {csv_path}, {json_path}")
    print(df.head())
    print(df.shape)

if __name__ == "__main__":
    main(CONFIG)

[WARN] No login → scraping public jobs only
[INFO] Fetched 30 jobs for Data Analyst in Pune, Maharashtra, India
[INFO] Fetched 30 jobs for Data Analyst in Bengaluru, Karnataka, India
[AUTO-SAVE] Saved 60 jobs → linkedin_jobs_20250816_105146.csv, linkedin_jobs_20250816_105146.json
[INFO] Fetched 30 jobs for Software Engineer in Pune, Maharashtra, India
[AUTO-SAVE] Saved 88 jobs → linkedin_jobs_20250816_105146.csv, linkedin_jobs_20250816_105146.json
[INFO] Fetched 30 jobs for Software Engineer in Bengaluru, Karnataka, India
[AUTO-SAVE] Saved 118 jobs → linkedin_jobs_20250816_105146.csv, linkedin_jobs_20250816_105146.json
[SUMMARY] Total scraped: 120, Unique after dedupe: 118
Saved → linkedin_jobs_20250816_105146.csv, linkedin_jobs_20250816_105146.json
                                               Title  \
0                                            Analyst   
1                                          Associate   
2                                       Data Analyst   
3  Senior Analys